In [ ]:
import os
import spacy
import pandas as pd
import networkx as nx
import pyvis

def locate_names(sentence: list, names: pd.DataFrame) -> list:
    # Function that returns names found in sentence

    stuff = []
    for token in sentence: 
        if token.text in list(names["First"]) or token.text in list(names["Last"]):
            stuff.append(token.text) 
    return stuff

    return [e for e in entities if e in list(names["First"]) or e in list(names["Last"])]

# Some important characters are called/known by multiple aliases
# Some like kings, Amyrlins, Nae'blis change with time
# Some are necessary because of transformations, but others may be omitted because familiar usage implies closeness or importance
aliases = {
    "Rand al'Thor" : ["Lews Therin Telamon", "Dragon Reborn", "Car'a'carn"],
    "Ishamael" : ["Elan Morin Tedronai", "Moridin", "Ba'alzamon"],
    "Dark One" : ["Shai'tan", "Father of Lies", "Sightblinder", "Lord of the Grave", "Great Lord"],
    "Aginor" : ["Osan'gar"],
    "Lanfear" : ["Cyndane"],
    "Mazrim Taim" : ["M'Hael"],
    "Balthamel" : ["Aran'gar", "Halima", "Corlan Dashiva"],
    "Asmodean" : ["Jasin Natael"],
    "Zarine Basher" : ["Faile"],
    "Galadedrid Damodred" : ["Galad"],
    "Someshta" : ["The Green Man"]
}

# Convert scraped text file to dataframe
df = pd.read_csv("final_names.txt", header = None)
df.columns = ["Name"]
# Split names into first and last
df[["First", "Last"]] = df["Name"].str.split(" ", n = 1, expand = True)

content = '''Rand al'Thor wandered the palace, deftly keeping his balance when the earth heaved. "Ilyena! 
My love, where are you?" The edge of his pale gray cloak trailed through blood as he stepped across the 
body of a woman, her golden-haired beauty marred by the horror of her last moments, her still-open 
eyes frozen in disbelief. "Where are you, my wife? Where is everyone hiding?" '''

# Read actual book text
with open("data/eyeoftheworld.txt", "r", encoding = "utf-8") as f:
    content = f.read()[:100000]
    print(content)

nlp = spacy.load("en_core_web_sm")
doc = nlp(content)

# todo: get rid of first/last name duplicates
# display node degrees
# for ego graph use weights, sentence differences
edges = []
for token in doc.sents:
    ppl = locate_names(token, df)
    if len(ppl) > 0:
        edges += ppl

# create tuples for edge list.
edges = [(x, y) for x in edges for y in edges if x != y]

# construct graph from edge list, and visualize
G = nx.Graph(edges)
nt = pyvis.network.Network("800px", "800px", select_menu=True, notebook=True, cdn_resources="remote")
nt.repulsion()
nt.from_nx(G)
nt.show("graph.html")